In [1]:
!python demo.py 

Traceback (most recent call last):
  File "C:\Users\chris\Google Drive\My Drive\Uni\MA\pytorch-YOLOv4\demo.py", line 162, in <module>
    detect_cv2(args.cfgfile, args.weightfile, args.imgfile)
  File "C:\Users\chris\Google Drive\My Drive\Uni\MA\pytorch-YOLOv4\demo.py", line 28, in detect_cv2
    m = Darknet(cfgfile)
  File "C:\Users\chris\Google Drive\My Drive\Uni\MA\pytorch-YOLOv4\tool\darknet2pytorch.py", line 130, in __init__
    self.blocks = parse_cfg(cfgfile)
  File "C:\Users\chris\Google Drive\My Drive\Uni\MA\pytorch-YOLOv4\tool\config.py", line 9, in parse_cfg
    line = fp.readline()
OSError: [Errno 28] No space left on device


In [11]:
import collections
  
# initializing deque
de = collections.deque('a')
de.append(4)
print("deque: ", de)

deque:  deque(['a', 4])


In [2]:
cd pytorch-YOLOv4

[WinError 2] The system cannot find the file specified: 'pytorch-YOLOv4'
C:\Users\chris\Google Drive\My Drive\Uni\MA\pytorch-YOLOv4


In [3]:
cd ..

C:\Users\chris\Google Drive\My Drive\Uni\MA


In [4]:
cd pytorch-mobilenet-v2

C:\Users\chris\Google Drive\My Drive\Uni\MA\pytorch-mobilenet-v2


In [5]:
ls

 Volume in drive C is OS
 Volume Serial Number is F2E9-03BF

 Directory of C:\Users\chris\Google Drive\My Drive\Uni\MA\pytorch-mobilenet-v2

10/29/2022  01:00 PM    <DIR>          .
10/29/2022  11:00 AM    <DIR>          ..
10/30/2022  02:12 AM    <DIR>          .ipynb_checkpoints
10/29/2022  10:17 AM    <DIR>          __pycache__
10/29/2022  09:45 AM            11,558 LICENSE
10/29/2022  01:00 PM             5,139 MobileNetV2.py
10/29/2022  09:45 AM             2,469 README.md
               3 File(s)         19,166 bytes
               4 Dir(s)     173,703,168 bytes free


In [6]:
import torch.nn as nn
import math


def conv_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU6(inplace=True)
    )


def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU6(inplace=True)
    )


def make_divisible(x, divisible_by=8):
    import numpy as np
    return int(np.ceil(x * 1. / divisible_by) * divisible_by)


class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio):
        super(InvertedResidual, self).__init__()
        self.stride = stride
        assert stride in [1, 2]

        hidden_dim = int(inp * expand_ratio)
        self.use_res_connect = self.stride == 1 and inp == oup

        if expand_ratio == 1:
            self.conv = nn.Sequential(
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )
        else:
            self.conv = nn.Sequential(
                # pw
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)


class MobileNetV2(nn.Module):
    def __init__(self, n_class=1000, input_size=224, width_mult=1.):
        super(MobileNetV2, self).__init__()
        block = InvertedResidual
        input_channel = 32
        last_channel = 1280
        interverted_residual_setting = [
            # t, c, n, s
            [1, 16, 1, 1],
            [6, 24, 2, 2],
            [6, 32, 3, 2],
            [6, 64, 4, 2],
            [6, 96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1],
        ]

        # building first layer
        assert input_size % 32 == 0
        # input_channel = make_divisible(input_channel * width_mult)  # first channel is always 32!
        self.last_channel = make_divisible(last_channel * width_mult) if width_mult > 1.0 else last_channel
        self.features = [conv_bn(3, input_channel, 2)]
        # building inverted residual blocks
        for t, c, n, s in interverted_residual_setting:
            output_channel = make_divisible(c * width_mult) if t > 1 else c
            for i in range(n):
                if i == 0:
                    self.features.append(block(input_channel, output_channel, s, expand_ratio=t))
                else:
                    self.features.append(block(input_channel, output_channel, 1, expand_ratio=t))
                input_channel = output_channel
        # building last several layers
        self.features.append(conv_1x1_bn(input_channel, self.last_channel))
        # make it nn.Sequential
        self.features = nn.Sequential(*self.features)

        # building classifier
        self.classifier = nn.Linear(self.last_channel, n_class)

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = x.mean(3).mean(2)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                n = m.weight.size(1)
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

    def mobilenet_v2(pretrained=True):
        model = MobileNetV2(width_mult=1)

        if pretrained:
            try:
                from torch.hub import load_state_dict_from_url
            except ImportError:
                from torch.utils.model_zoo import load_url as load_state_dict_from_url
            state_dict = load_state_dict_from_url(
                'https://www.dropbox.com/s/47tyzpofuuyyv1b/mobilenetv2_1.0-f2a8633.pth.tar?dl=1', progress=True)
            model.load_state_dict(state_dict)
        return model



In [7]:
from tool.MobileNetV2 import *

def test():
    model_url = 'https://raw.githubusercontent.com/d-li14/mobilenetv2.pytorch/master/pretrained/mobilenetv2-c5e733a8.pth'
    net = mobilenetv2(model_url)
    #print(net)
    x = torch.randn(2,3,224,224)
    y1,y2 = net(x)
    print(y2.shape, net)
    
test()

NameError: name 'torch' is not defined

In [ ]:
!pip install easydict

In [ ]:
!pip install pycocotools

In [ ]:
!pip install tensorboardX

In [ ]:
!python trainmnv2.py -g 0 -dir r"Google Drive/My Drive/Uni/MA/coco/train2017"

In [ ]:

import os
_BASE_DIR = os.path.dirname(os.path.abspath(__file__))
os.path.join(_BASE_DIR,'..', '..' )



In [ ]:
from pathlib import Path
import os
src = Path.cwd()
#os.path.split(src, os.path.splitext(os.path.basename(src))[0])
#print(src)
sfle, rfle = os.path.split(src)

print(sfle)


In [ ]:
def test():
    model_url = 'https://raw.githubusercontent.com/d-li14/mobilenetv2.pytorch/master/pretrained/mobilenetv2-c5e733a8.pth'
    net = mobilenetv2(model_url)
    #print(net)
    x = torch.randn(2,3,224,224)
    y1,y2 = net(x)
    print(y2.shape, net)
    
test()